# tweet grab

In [12]:
import os
import tweepy
import logging
import pprint
import json
import time
import datetime as dt
import pandas as pd
import numpy as np

import boto3
import botocore.session
from botocore.exceptions import ClientError
from boto3.dynamodb.conditions import Key, Attr

print('imported modules successfully.')

imported modules successfully.


In [2]:
! pip3 install ../main/dist/twitter-0.1.0-py3-none-any.whl --force-reinstall

Processing /home/steven/Documents/twitter.ai/main/dist/twitter-0.1.0-py3-none-any.whl
  Attempting uninstall: twitter
    Found existing installation: twitter 0.1.0
    Uninstalling twitter-0.1.0:
      Successfully uninstalled twitter-0.1.0


In [3]:
# import twitter

### set api creds

In [4]:
base = os.getcwd()
print(base)

/home/steven/Documents/twitter.ai/notebooks


In [13]:
base = os.getcwd()
data_dir = os.path.join(base, 'data')
log_dir = os.path.join(base, 'logs')
pp = pprint.PrettyPrinter(indent = 1)

os.chdir(base)

with open('./../creds.json', 'r') as f:
    tweepy_creds = json.load(f)
    f.close()
    
auth = tweepy.OAuthHandler(tweepy_creds['twitter-api-key'], tweepy_creds['twitter-secret-key'])
auth.set_access_token(tweepy_creds['twitter-access-token'], tweepy_creds['twitter-secret-access'])

api = tweepy.API(auth)
logging.debug('authorized API w/ client id and secret.')

try:
  client = boto3.client('dynamodb',
    endpoint_url = "http://{}:{}".format(tweepy_creds["host"], tweepy_creds["port"]),
    aws_access_key_id=tweepy_creds["aws-access-key"],
    aws_secret_access_key=tweepy_creds["aws-secret-access-key"],
    region_name='us-east-2')

  dynamodb = boto3.resource('dynamodb', \
              endpoint_url = "http://{}:{}".format(tweepy_creds["host"], tweepy_creds["port"]), \
              region_name='us-east-2')

  session = botocore.session.get_session()
  dynamodb_session = session.create_client('dynamodb', \
                      region_name='us-east-2',
                      endpoint_url = "http://{}:{}".format(tweepy_creds["host"], tweepy_creds["port"]),
                      aws_access_key_id=tweepy_creds["aws-access-key"],
                      aws_secret_access_key=tweepy_creds["aws-secret-access-key"]) # low-level client

  tweets = dynamodb.Table("tweets")

  print("created dynamo client.")

except Exception as e:
  print(e)

created dynamo client.


## get followers

In [14]:
def get_followers(user):
    following = []
    friends_response = api.get_followers(screen_name = user, count = 100, cursor = -1)
    following.extend([t.screen_name for t in friends_response[0]])
    next_cursor = friends_response[1][1]
    print("got most recent 100 following")

    while next_cursor != 0:
        try:
            friends_response = api.get_followers(screen_name = user, count = 100, cursor = next_cursor)
            following.extend([t.screen_name for t in friends_response[0]])
            next_cursor = friends_response[1][1]
            print("got next 100 following")
        
        except:
            time.sleep(60*15)

    return(following)

try:
    my_following = get_followers("xanax_princess_")

except Exception as e:
    print("Exception: {}".format(e))

print(len(my_following))

got most recent 100 following
got next 100 following
198


## batch put response

In [15]:
def batch_put_response(response, table):
    with table.batch_writer(overwrite_by_pkeys=["user", "created_at"]) as batch:
        for r in response:
            try:
                batch.put_item(Item = {
                    "user": r.user.screen_name,
                    "created_at": r.created_at.strftime("%Y-%m-%dT%H:%M:%SZ"),
                    "tweet": r.text,
                    "id": r.id,
                    "retweets": r.retweet_count,
                    "favorites": r.favorite_count,
                    "inserted_at": dt.datetime.now().strftime("%Y-%m-%dT%H:%M:%SZ")
                })
        
            except ClientError as err:
                print("Couldn't load data into table {}. Here's why: {}: {}".format(table.name, err.response['Error']['Code'], err.response['Error']['Message']))
                raise

### get tweets per user

In [28]:
def get_tweets(user, table):
    print("scraping tweets for user: {}".format(user))
    timeline_response = api.user_timeline(screen_name = user, count = 25)
    n = len(timeline_response) - 2
    try:
        last_id = timeline_response[n-1]._json["id"]
    
    except Exception as e:
        print(e)
        # last_id = timeline_response[24]._json["id"]
        print("count: {}.".format(n))
        
    i = 0

    print("putting first batch")
    batch_put_response(timeline_response, table)
    print("inserted first batch to dynamodb successfully.")
    i += 1

    while i < 25:
        try:
            timeline_response = api.user_timeline(screen_name = user, count = 25, max_id = last_id)
            n = len(timeline_response) - 2
            try:
                last_id = timeline_response[n-1]._json["id"]

            except Exception as e:
                print(e)
                last_id = timeline_response[24]._json["id"]

            batch_put_response(timeline_response, table)
            i += 1

            if (i > 0 and (i % 5) == 0):
                print("inserted batch {} to dynamodb successfully.".format(i+1))
    
        except Exception as e:
            print(e)
            if "429" in str(e):
                print("sleeping for 5 minutes")
                time.sleep(60*5)

            else:
                print("sleeping for a minute")
                time.sleep(60)

In [29]:
# timeline_response = api.user_timeline(screen_name = my_following[0], count = 100)
# my_id = timeline_response[0]._json["id"]

In [30]:
# timeline_response = api.user_timeline(screen_name = my_following[0], count = 25, max_id = my_id)

In [31]:
# timeline_response[23]._json["id"]

## get all tweets

In [32]:
tweets = dynamodb.Table("tweets")
print(tweets)

dynamodb.Table(name='tweets')


## get all users

In [33]:
try:
    response = tweets.scan()
    users = []

    for r in response['Items']:
        users.append(r['user'])

    unique_users = np.unique(users)
    print(unique_users)

except Exception as e:
    print(e)

['kittyxcate']


In [13]:
# my_following

In [14]:
# my_ind = [f not in unique_users for f in my_following]

# for i, f in enumerate(my_following):
#     if my_ind[i]:
#         try:
#             output = get_tweets(f, tweets)
#             time.sleep(5)

#         except KeyboardInterrupt as k:
#             print(k)

#     else:
#         pass

## get tweets of following

In [15]:
for f in my_following:
    try:
        output = get_tweets(f, tweets)
        time.sleep(5)

    except KeyboardInterrupt as k:
        print(k)

scraping tweets for user: kittyxcate
putting first batch
inserted first batch to dynamodb successfully.
inserted batch 6 to dynamodb successfully.
inserted batch 11 to dynamodb successfully.
inserted batch 16 to dynamodb successfully.
inserted batch 21 to dynamodb successfully.
inserted batch 26 to dynamodb successfully.
scraping tweets for user: coy_valerie23
list index out of range
count: 0.
putting first batch
inserted first batch to dynamodb successfully.
local variable 'last_id' referenced before assignment
sleeping for a minute


In [37]:
response = tweets.scan()
response

{'Items': [{'favorites': Decimal('0'),
   'created_at': '2021-12-10T07:27:40Z',
   'tweet': 'If another man farts in my car I’m gonna deck the shit outta them',
   'id': Decimal('1469207171246215171'),
   'retweets': Decimal('0'),
   'user': 'kittyxcate',
   'inserted_at': '2023-02-04T20:48:52Z'},
  {'favorites': Decimal('2'),
   'created_at': '2021-12-10T07:31:09Z',
   'tweet': 'Happiest of birthdays to my angel bean @_gutmutt love love love u friend I’m glad u were born into this lifetime 💝 https://t.co/rHUO7pvNtn',
   'id': Decimal('1469208048153595908'),
   'retweets': Decimal('1'),
   'user': 'kittyxcate',
   'inserted_at': '2023-02-04T20:48:52Z'},
  {'favorites': Decimal('0'),
   'created_at': '2021-12-10T07:33:50Z',
   'tweet': 'RT @puddlecore: we used to be a real country https://t.co/6rp60queWe',
   'id': Decimal('1469208723419803649'),
   'retweets': Decimal('2142'),
   'user': 'kittyxcate',
   'inserted_at': '2023-02-04T20:48:52Z'},
  {'favorites': Decimal('0'),
   'created_

In [36]:
! aws dynamodb describe-table --table-name "tweets" --endpoint-url https://xanaxprincess.asuscomm.com:8000


SSL validation failed for https://xanaxprincess.asuscomm.com:8000/ [SSL: WRONG_VERSION_NUMBER] wrong version number (_ssl.c:997)
